In [ ]:
%matplotlib ipympl
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.ndimage import gaussian_filter
from scipy.spatial.distance import cdist

from vizlib import *

In [ ]:
filename = "/Volumes/DATA/amatter_24_01_2024_card_worms_only.xyzv"
N,nframes,pos_data,vel_data = load_data(filename)

In [ ]:
vector_field = get_particle_vecs_frame(N,-1,pos_data)

In [ ]:
vector_field.shape

In [ ]:
"""plotting the vectors of each of the individual particles"""
fig,ax = plt.subplots(1,1)
quiver = ax.quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax.set_box_aspect(1)
ax.set_adjustable("datalim")
plt.show()

In [ ]:
def local_average_director(data, radius):
    """
    Perform local averaging of the director data.

    Parameters:
    - data: (N, 6) array with columns for x-position, y-position, z-position, x-dir, y-dir, z-dir.
    - radius: Radius of the local neighborhood for averaging.

    Returns:
    - averaged_data: (N, 6) array with columns for x-position, y-position, z-position, averaged x-dir, averaged y-dir, averaged z-dir.
    """
    num_points = data.shape[0]
    averaged_data = np.zeros_like(data)
    empty_count = 0
    for i in range(num_points):
        # Calculate distances to all other points
        distances = cdist(data[:, :2], data[i, :2].reshape(1, -1))[0]

        # Identify points within the specified radius
        neighbors_indices = np.where(distances <= radius)[0]
        if neighbors_indices.size > 0:
            # Average the director components within the local neighborhood
            averaged_x_dir = np.mean(data[neighbors_indices, 3])
            averaged_y_dir = np.mean(data[neighbors_indices, 4])

            # Update the averaged_data array
            averaged_data[i, :] = [data[i, 0], data[i, 1], 0.0,averaged_x_dir, averaged_y_dir,0.0]
        else:
            averaged_data[i, :] = [data[i, 0], data[i, 1], 0.0, data[i,3], data[i,4], 0.0]
            empty_count += 1
    print("Points with no neighbors (increase radius?):",empty_count,"/",num_points,"\t",(empty_count/num_points)*100,"%")
    return averaged_data

radius = 100.0  # You can adjust the radius based on your simulation requirements
averaged_data = local_average_director(vector_field, radius)


In [ ]:
"""plotting the vectors of each of the averaged particle data"""
fig,(ax1, ax2) = plt.subplots(1, 2,figsize=(12,6))
quiver = ax1.quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax1.set_box_aspect(1)
ax1.set_adjustable("datalim")
ax1.set_title('Filament Vectors')
quiver = ax2.quiver(averaged_data[:,0], #x
                   averaged_data[:,1], #y
                   averaged_data[:,3], #u
                   averaged_data[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax2.set_box_aspect(1)
ax2.set_title('Locally-Averaged Directors')
ax2.set_adjustable("datalim")
plt.show()


In [ ]:
import pyinterp
mesh_x = pyinterp.RTree()
mesh_x.packing(np.vstack((vector_field[:,0], vector_field[:,1])).T, vector_field[:,3])
STEP = 1 / 32
max_x = np.max(vector_field[:,0])
min_x = np.min(vector_field[:,0])
max_y = np.max(vector_field[:,1])
min_y = np.min(vector_field[:,1])
mx, my = np.meshgrid(np.arange(min_x, max_x + STEP, STEP),
                        np.arange(min_y, max_x + STEP, STEP),
                        indexing='ij')
wf_x, neighbors = mesh.window_function(
    np.vstack((mx.ravel(), my.ravel())).T,
    within=False,  # Extrapolation is forbidden
    k=11,
    wf='parzen',
    num_threads=0)
wf_x = wf_x.reshape(mx.shape)

mesh_y = pyinterp.RTree()
mesh_y.packing(np.vstack((vector_field[:,0], vector_field[:,1])).T, vector_field[:,4])
wf_y, neighbors = mesh.window_function(
    np.vstack((mx.ravel(), my.ravel())).T,
    within=False,  # Extrapolation is forbidden
    k=11,
    wf='parzen',
    num_threads=0)
wf_y = wf_y.reshape(mx.shape)


In [55]:
fig,ax = plt.subplots(1,1)
quiver = ax.quiver(mx,my,wf_x,wf_y,
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax.set_box_aspect(1)
ax.set_adjustable("datalim")
plt.show()